In [132]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

In [133]:
# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "X9EGwt5zbpg8o5xk"

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [134]:
# Grab all event type ids. This will return a list which we will iterate over to print out the id and the name of the sport
event_types = trading.betting.list_event_types()

sport_ids = pd.DataFrame({
    'Sport': [event_type_object.event_type.name for event_type_object in event_types],
    'ID': [event_type_object.event_type.id for event_type_object in event_types]
}).set_index('Sport').sort_index()

sport_ids

,ID
Sport,
American Football,6423
Australian Rules,61420
Baseball,7511
Basketball,7522
Boxing,6
Cricket,4
Cycling,11
Darts,3503
Esports,27454571


In [135]:
gh_racing_id = 4339
# Filter for just horse racing
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(days=1)).strftime("%Y-%m-%dT%TZ")
    }
)
greyhound_racing_filter

{'eventTypeIds': [4339],
 'marketCountries': ['GB'],
 'marketStartTime': {'to': '2022-06-07T12:20:03Z'}}

In [136]:
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)
# Create a DataFrame with all the events by iterating over each event object
gb_gh_events_today = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_gh_events],
    'Event ID': [event_object.event.id for event_object in gb_gh_events],
    'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
    'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
    'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
    'Market Count': [event_object.market_count for event_object in gb_gh_events]
})

gb_gh_events_today

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Yarmouth (F/C) 6th Jun,31508256,Yarmouth,GB,Europe/London,2022-06-06 17:19:00,12
1,Central Park (F/C) 6th Jun,31508225,Central Park,GB,Europe/London,2022-06-06 09:46:00,2
2,Suffolk Downs 6th Jun,31508226,Suffolk Downs,GB,Europe/London,2022-06-06 09:53:00,8
3,Nottingham 6th Jun,31508258,Nottingham,GB,Europe/London,2022-06-06 17:26:00,24
4,Perry Barr 6th Jun,31508195,Perry Barr,GB,Europe/London,2022-06-06 10:06:00,6
5,Sheffield 6th Jun,31508228,Sheffield,GB,Europe/London,2022-06-06 13:04:00,24
6,Perry Barr (F/C) 6th Jun,31508198,Perry Barr,GB,Europe/London,2022-06-06 10:06:00,3
7,Swindon 6th Jun,31508200,Swindon,GB,Europe/London,2022-06-06 10:13:00,6
8,Sheffield (F/C) 6th Jun,31508233,Sheffield,GB,Europe/London,2022-06-06 13:04:00,12
9,Monmore 6th Jun,31508234,Monmore,GB,Europe/London,2022-06-06 12:49:00,28


In [137]:
venueOfTheDay = gb_gh_events_today['Event ID'][4]
venueOfTheDay
# market_types_filter = filters.market_filter(event_ids=[gb_gh_events_today[0].id])

'31508195'

In [138]:
market_types_filter = filters.market_filter(event_ids=[venueOfTheDay])

market_types = trading.betting.list_market_types(
    filter=market_types_filter
)

market_types_venueOfTheDay = pd.DataFrame({
    'Market Type': [market_type_object.market_type for market_type_object in market_types],
})
market_types_venueOfTheDay

,Market Type
0,PLACE
1,WIN


In [139]:
market_catalogue_filter = filters.market_filter(event_ids=[venueOfTheDay])

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='100',
    sort='FIRST_TO_START'
)

market_types_venueOfTheDay = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
})
market_types_venueOfTheDay

,Market Name,Market ID,Total Matched
0,To Be Placed,1.199935417,0.000000
1,A3 480m,1.199935416,86.945208
2,D4 275m,1.199935418,27.833600
3,To Be Placed,1.199935419,0.000000
4,A7 480m,1.199935420,0.000000
5,To Be Placed,1.199935421,0.000000


In [140]:
myRaceID = market_types_venueOfTheDay['Market ID'][0]
myRaceID

'1.199935417'

In [141]:
def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [142]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

runners_df

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,45676310,2.24,34.32,2.30,27.75,2.26,0.0,ACTIVE,None,None
1,42466510,4.50,49.97,5.10,33.12,4.60,0.0,ACTIVE,None,None
2,45305630,2.42,22.09,2.50,31.26,2.50,0.0,ACTIVE,None,None
3,38918289,2.44,20.87,2.56,22.46,2.50,0.0,ACTIVE,None,None
4,36799671,2.50,42.21,2.64,19.67,2.60,0.0,ACTIVE,None,None
5,36246164,5.60,30.21,6.80,32.18,6.40,0.0,ACTIVE,None,None


In [143]:
fav_selection_id = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Selection ID']
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

In [144]:
limit_order_filter = filters.limit_order(
    size=5,
    price=3,
    persistence_type='LAPSE'
)

instructions_filter = filters.place_instruction(
    selection_id = str(fav_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '45676310',
 'side': 'LAY',
 'limitOrder': {'price': 3, 'persistenceType': 'LAPSE', 'size': 5}}

In [145]:
# order = trading.betting.place_orders(
#     market_id = myRaceID,
#     customer_strategy_ref='Naing_maker',
#     instructions=[instructions_filter]
# )

In [146]:
# order.__dict__

In [147]:
# trading.betting.list_current_orders(customer_strategy_refs=['Naing_maker']).__dict__

In [148]:
# cancelled_order = trading.betting.cancel_orders(market_id=myRaceID)

In [149]:
# # listClearedOrders
# cleared_orders = trading.betting.list_cleared_orders(bet_status="CANCELLED",
#                                                     market_ids=[myRaceID])
# # Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['cancelledOrders'])
# cleared_orders